# **1. Veri İşleme** #

# - Kütüphaneler #

Projede kullanılacak kütüphanelerin dahil edilmesi.

In [ ]:
# Gerekli kütüphaneleri yüklüyoruz
import numpy as np  # Sayısal hesaplamalar için NumPy kütüphanesini içe aktarıyoruz
import pandas as pd  # Veri analizi ve manipülasyonu için Pandas kütüphanesini içe aktarıyoruz
import matplotlib.pyplot as plt  # Görselleştirme için Matplotlib kütüphanesini içe aktarıyoruz
from sklearn.model_selection import train_test_split  # Veri setini eğitim ve test olarak ayırmak için fonksiyonu içe aktarıyoruz
from keras.preprocessing.image import load_img, img_to_array  # Görüntü işleme için gerekli fonksiyonları içe aktarıyoruz
from tqdm import tqdm  # İlerleme çubuğu oluşturmak için kütüphaneyi içe aktarıyoruz
import seaborn as sns  # Gelişmiş veri görselleştirme için Seaborn kütüphanesini yüklüyoruz
from sklearn.metrics import classification_report, confusion_matrix  # Model performansını değerlendirmek için metrikleri içe aktarıyoruz
import tensorflow as tf  # TensorFlow kütüphanesini içe aktarıyoruz
import os  # İşletim sistemi ile etkileşim için os kütüphanesini içe aktarıyoruz
from tensorflow.keras.models import Sequential  # Model oluşturmak için Sequential sınıfını içe aktarıyoruz
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout  # CNN katmanlarını içe aktarıyoruz
from tensorflow.keras.optimizers import Adam  # Adam optimizasyon algoritmasını içe aktarıyoruz
import warnings  # Uyarı yönetimi için warnings kütüphanesini içe aktarıyoruz

warnings.filterwarnings("ignore")  # Uyarıları gizlemek için kullanıyoruz

# - Veri setini Tanımlama #

Verilerin okunması ve data_directory'e aktarılması.

In [ ]:
# Resim dosyalarının bulunduğu ana dizin
data_directory = '//kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'

# Bu dizindeki alt klasörler (balık sınıfları) alınıyor. '.' içermeyen klasörler sınıf isimleridir.
fish_classes = [folder for folder in os.listdir(data_directory) if '.' not in folder]
fish_classes  # Bulunan balık türlerini listele

In [ ]:
# Resim dosyalarının yolları ve etiketleri için başlangıçta boş listeler oluşturuluyor
image_labels = []  # Resim etiketleri için liste
file_paths = []  # Resim dosya yolları için liste

# Belirtilen dizindeki dosya yapısını gezerek işlemler yapıyoruz
for directory, _, files in os.walk(data_directory):
    for file in files:
        # Sadece '.png' uzantılı dosyaları işleme alıyoruz
        if file.endswith('.png'):
            # Eğer 'GT' (ground truth) klasöründe değilse etiketi al
            if 'GT' not in os.path.basename(directory):            
                image_labels.append(os.path.basename(directory))  # Klasör ismini etiketi olarak ekle
                file_paths.append(os.path.join(directory, file))  # Dosya yolunu listeye ekle

# Elde edilen resim yolları ve etiketler ile bir DataFrame oluşturuyoruz
dataset = pd.DataFrame(columns=['path', 'label'])  # Boş bir DataFrame oluştur
dataset['path'] = file_paths  # Resim yollarını ekliyoruz
dataset['label'] = image_labels  # Etiketleri ekliyoruz

In [ ]:
dataset

# - Birkaç Satır ve Temel Bilgileri Gösterme #

In [ ]:
# İlk resim yolunu görüntülüyoruz (index 0'daki)
first_image_path = dataset.path.iloc[0]  # .iloc kullanarak ilk resim yolunu alıyoruz
first_image_path

In [ ]:
# Veri setinin yapısına göz atalım
print(dataset.info())

# Veri setinden örnek satırlar görüntüleyerek genel yapısına dair bir fikir edinelim
sample_data = dataset.head()  # İlk 5 satırı seçiyoruz
print(sample_data)

# Her balık sınıfının veri setindeki sayısını belirleyelim
# Bu, her etiketin ne kadar temsil edildiğini anlamamıza yardımcı olur
fish_class_distribution = dataset['label'].value_counts()
print("Numbers of fish classes:\n", fish_class_distribution)


# - Veri Görselleştirme #

Veri setindeki türlerin dağılımı görsel olarak sınıflandırılmıştır.

In [ ]:
# Balık türlerinin dağılımını görsel olarak inceleyelim
plt.figure(figsize=(12, 8))  # Görüntü boyutunu ayarlıyoruz
sns.countplot(data=dataset, x='label', palette='Spectral')  # Çubuk grafikle sınıf sayılarını gösteriyoruz
plt.title('Fish Species Class Distribution', fontsize=16, fontweight='bold')  # Grafik başlığını kalın yapıyoruz
plt.xlabel('Fish Species', fontsize=12)  # X eksenine etiket ekliyoruz
plt.ylabel('Number', fontsize=12)  # Y eksenine etiket ekliyoruz
plt.xticks(rotation=30, ha='right', fontsize=10)  # X eksenindeki etiketleri döndürerek hizalıyoruz
plt.yticks(fontsize=10)  # Y ekseni etiket boyutunu ayarlıyoruz
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Y eksenine kılavuz çizgileri ekliyoruz
plt.tight_layout()  # Grafik düzenini iyileştiriyoruz
plt.show()  # Grafiği ekranda gösteriyoruz

Her sınıftan birkaç örnek gösterme.

In [ ]:
# Her sınıftan birkaç örnek görüntüyü görselleştirelim
plt.figure(figsize=(24, 24))  # Grafik boyutunu ayarlıyoruz
# İlk 9 balık sınıfını gösterecek şekilde döngü oluşturuyoruz
for i, label in enumerate(dataset['label'].unique()[:9]):  
    plt.subplot(4, 3, i + 1)  # 4x3'lük alt grafik oluşturuyoruz
    # İlgili sınıfa ait ilk görüntüyü yüklüyoruz
    image_path = dataset[dataset['label'] == label].iloc[0, 0]  
    img = plt.imread(image_path)
    plt.imshow(img)  # Resmi yüklüyoruz
    plt.title(label, fontsize=18, fontweight='bold')  # Sınıf adını başlık olarak ekliyoruz
    plt.axis('off')  # Eksenleri kapatıyoruz

plt.suptitle('Sample Images of Fish Species', fontsize=20, fontweight='bold')  # Üst başlık ekliyoruz
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Alt grafiklerin düzenini iyileştiriyoruz
plt.show()  # Grafiği gösteriyoruz

# **2. Model** #

In [ ]:
# Veri setini eğitim ve test olarak ayırıyoruz.
train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42, stratify=dataset['label'])

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    try:
        img = load_img(image_path, target_size=target_size)  # Görüntüyü belirtilen boyutta yüklüyoruz
        img_array = img_to_array(img)  # Görüntüyü dizi formatına çeviriyoruz
        return img_array / 255.0  # Piksel değerlerini normalize ediyoruz
    except Exception as e:
        print(f"Error: {e}. {image_path} could not be loaded.")
        return None #Hata durumunda mesaj göster

# Eğitim verileri için X_train ve y_train hazırlıyoruz
X_train = np.array([img for img in (load_and_preprocess_image(path) for path in tqdm(train_set['path'], desc="Loading Training Images")) if img is not None])
y_train = pd.get_dummies(train_set['label']).values  # Etiketleri one-hot encoding ile dönüştürüyoruz

# Test verileri için X_test ve y_test hazırlıyoruz
X_test = np.array([img for img in (load_and_preprocess_image(path) for path in tqdm(test_set['path'], desc="Loading Test Images")) if img is not None])
y_test = pd.get_dummies(test_set['label']).values  # Test etiketlerini one-hot encoding ile dönüştürüyoruz

# Eğitim ve test setlerinin boyutlarını yazdırıyoruz
print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

# **3. Modelin Tanımlanması** #

In [ ]:
# Model oluşturma
model = Sequential()

# İlk Convolutional Katman
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(2, 2))

# İkinci Convolutional Katman
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Üçüncü Convolutional Katman
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Flatten Katmanı
model.add(Flatten())

# Tam Bağlantılı Katmanlar
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Çıkış Katmanı (Sınıf sayısına göre)
model.add(Dense(9, activation='softmax'))  # Sınıf sayısı dataset'e göre ayarlanmış

# Modeli derlemek
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model özeti
model.summary()

# **4. Modelin Eğitilmesi ve Geliştirilmesi**  #

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Model eğitimi
history = model.fit(
    X_train,                # Eğitim verileri
    y_train,                # Eğitim etiketleri
    epochs=20,              # Eğitim döngü sayısı (20 epoch ile başlatılıyor)
    batch_size=32,          # Her iterasyonda işlenecek görüntü sayısı
    validation_split=0.2,   # Eğitim verilerinin %20'si doğrulama için ayrılacak
    callbacks=[early_stopping]  # Erken durdurma ile aşırı öğrenmeyi engelleme
)

# - Accuracy ve Loss Grafiği #

Eğitim kayıplarını ve doğruluğu görselleştirelim 

In [ ]:
# Kayıp (Loss) grafiği
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Graph')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Doğruluk (Accuracy) grafiği
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Graph')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# - Modelin Değerlendirilmesi #

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

In [ ]:
# Modelin test verileri üzerindeki tahminleri
y_pred_probs = model.predict(X_test)  # Tahmin edilen olasılıklar
y_pred_classes = np.argmax(y_pred_probs, axis=1)  # En yüksek olasılığa sahip sınıfı seçiyoruz

# Gerçek sınıfları elde ediyoruz
y_true_classes = np.argmax(y_test, axis=1)  # One-hot encoding'den orijinal sınıf etiketlerine dönüştürüyoruz

In [ ]:
from sklearn.metrics import classification_report

# Sınıflandırma raporu
print(classification_report(y_true_classes, y_pred_classes, target_names=dataset['label'].unique()))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Karışıklık matrisi
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Karışıklık matrisini görselleştirme
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=dataset['label'].unique(), yticklabels=dataset['label'].unique())
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# **5. Veri Setindeki Görüntüyü Sınıflandırma ve Tahmin İşlemi** #

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Örnek bir resim yükleme (Pillow ile)
image_path = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset/Hourse Mackerel/Hourse Mackerel/00030.png'

# Görüntüyü Pillow ile aç, boyutlandır ve normalize et
test_img = Image.open(image_path)  # Görüntüyü aç
test_img = test_img.resize((224, 224))  # Modelin beklediği boyut (224x224) olacak şekilde yeniden boyutlandır
test_img_array = np.array(test_img).astype('float32') / 255  # NumPy dizisine dönüştür ve normalize et

# Modelin beklediği formatta giriş oluşturmak için boyutları genişlet
test_img_array = np.expand_dims(test_img_array, axis=0)  # (1, 224, 224, 3) boyutuna genişlet

# Modelden tahmin yapma
predictions = model.predict(test_img_array)

# Tahmin edilen sınıf indeksini bulma
predicted_class_index = np.argmax(predictions)

# Dataset içindeki etiket isimlerini alıyoruz
class_names = dataset['label'].unique()  # Sınıf isimlerini elde ediyoruz

# Tahmin edilen sınıf ismi
predicted_class = class_names[predicted_class_index]

# Tahmin sonucunu görselleştirme (Pillow ile)
plt.imshow(test_img)  # test_img PIL imaj nesnesidir, NumPy dizisi değil
plt.title(f'Predicted Class: {predicted_class}')
plt.axis('off')  # Eksenleri gizle
plt.show()